In [646]:
using StructuralIdentifiability 
using Nemo
using IterTools


In [647]:
ode = @ODEmodel(
    x1'(t) = x1(t) ,
    x2'(t) = x1(t) - a*x2(t) * x2(t) ,
    x3'(t) = x3(t) - x1(t) * x2(t) * x3(t) ,
    y1(t) = x1(t),
    y2(t) = x1(t) + x2(t),
    y3(t) = x3(t)
)

ode2 = @ODEmodel(
    x1'(t) = x1(t) +a*x2(t)^2,
    x2'(t) = x2(t) - x1(t) * x2(t) ,
    x3'(t) = x3(t) + x1(t) + x2(t),
    y1(t) = x1(t),
    y2(t) = x3(t)
)

ode3 = @ODEmodel(
    o1'(t) = r1*o1(t),
    x0'(t) = o1(t) - d*x0(t),
    x1'(t) = x0(t) - x1(t),
    x2'(t) = x1(t) - x2(t),
    x3'(t) = x2(t) - x3(t),
    x4'(t) = x3(t) - x4(t),
    x5'(t) = x4(t) - x5(t),
    o12'(t) = r2*o12(t),
    x02'(t) = o12(t) + x2(t) + x3(t) + x22(t) + x32(t) + x02(t),
    x12'(t) = x2(t) + x3(t) + x22(t) + x32(t) + x02(t) - x12(t),
    x22'(t) = x12(t) - x22(t),
    x32'(t) = x22(t) - x32(t),
    x42'(t) = x32(t) - x42(t),
    x52'(t) = x42(t) - x52(t),
    y0(t) = x0(t) + x02(t),
    y1(t) = x1(t) + x12(t),
    y2(t) = x2(t) + x22(t),
    y3(t) = x3(t) + x32(t),
    y4(t) = x4(t) + x42(t) + x5(t) + x52(t),
    y5(t) = x1(t) + x2(t) + x3(t) + x4(t) + x12(t) + x22(t) + x32(t) + x42(t) ,
    y6(t) = x5(t) + x52(t) 
)

ode4 = @ODEmodel(
    x1'(t) = x1(t),
    x2'(t) = x2(t),
    x3'(t) = x3(t),
    x4'(t) = x4(t),
    x5'(t) = x5(t),
    x6'(t) = x6(t),
    x7'(t) = x7(t),
    x8'(t) = x8(t),
    x9'(t) = x9(t),
    x10'(t) = x10(t),
    x11'(t) = x11(t),
    x12'(t) = x12(t),
    x13'(t) = x13(t),
    x14'(t) = x14(t),
    x15'(t) = x15(t),
    x16'(t) = x16(t),
    x17'(t) = x17(t),
    y1(t) = x1(t),
    y2(t) = x2(t),
    y3(t) = x3(t),
    y4(t) = x4(t),
    y5(t) = x5(t),
    y6(t) = x6(t),
    y7(t) = x7(t),
    y8(t) = x8(t),
    y9(t) = x9(t),
    y10(t) = x10(t),
    y11(t) = x11(t),
    y12(t) = x12(t),
    y13(t) = x13(t),
    y14(t) = x14(t),
    y15(t) = x15(t),
    y16(t) = x16(t),
    y17(t) = x17(t)
    
)

println(1)

1


┌ Info: Summary of the model:
└ @ StructuralIdentifiability /Users/stefanvayl/.julia/packages/StructuralIdentifiability/egNv7/src/ODE.jl:329
┌ Info: State variables: x2, x3, x1
└ @ StructuralIdentifiability /Users/stefanvayl/.julia/packages/StructuralIdentifiability/egNv7/src/ODE.jl:330
┌ Info: Parameters: a
└ @ StructuralIdentifiability /Users/stefanvayl/.julia/packages/StructuralIdentifiability/egNv7/src/ODE.jl:331
┌ Info: Inputs: 
└ @ StructuralIdentifiability /Users/stefanvayl/.julia/packages/StructuralIdentifiability/egNv7/src/ODE.jl:332
┌ Info: Outputs: y3, y1, y2
└ @ StructuralIdentifiability /Users/stefanvayl/.julia/packages/StructuralIdentifiability/egNv7/src/ODE.jl:333
┌ Info: Summary of the model:
└ @ StructuralIdentifiability /Users/stefanvayl/.julia/packages/StructuralIdentifiability/egNv7/src/ODE.jl:329
┌ Info: State variables: x2, x3, x1
└ @ StructuralIdentifiability /Users/stefanvayl/.julia/packages/StructuralIdentifiability/egNv7/src/ODE.jl:330
┌ Info: Parameters: a
└ 

In [648]:

function construct_graph(ode) # construct a graph from given system of equations
    graph = Dict()
    for (x,f) in ode.x_equations
        graph[x] = vars(f)
    end
    for (y,f) in ode.y_equations
        graph[y] = vars(f)
    end
    return graph
end




construct_graph (generic function with 1 method)

In [649]:

function dfs(graph, start, visited) # dfs algo for traversing obtained graph
    push!(visited, start)
    if start in keys(graph)
        for node in graph[start]
            if !(node in visited)
                dfs(graph, node, visited)
            end 
        end
    end
    return visited
end



dfs (generic function with 1 method)

In [650]:


function traverse_outputs(graph, y)
    raw_models = Dict()
    for state in y
        model = dfs(graph, state, [])
        raw_models[state] = model
    end
    return raw_models
end


traverse_outputs (generic function with 1 method)

In [651]:

function find_raw_submodels(raw_models,Y, graph)
    result = []
    for y1 in Y
        for y2 in Y
            if y1 != y2
                if issubset(graph[y2], raw_models[y1]) && !(y2 in raw_models[y1])
                   push!(raw_models[y1], y2)
                end
            end
        end
    end
    for y in Y
        push!(result, raw_models[y])
    end
    return result
end


find_raw_submodels (generic function with 1 method)

In [652]:
function sort_all(submodels)
    sorted = []
    for submodel in submodels
        sub = sort(submodel, by = string)
        push!(sorted, sub)
    end
    return sorted
end

sort_all (generic function with 1 method)

In [653]:
function search_add_unions(submodels)
    result = [[]]
    for model in submodels
        for index in range(1,length(result))
            candidate_model = union(result[index], model)
            push!(result, candidate_model)
        end
    end
    return result
end
    

search_add_unions (generic function with 1 method)

In [654]:
function remove_empty(submodels)
    if [] in submodels
        deleteat!(submodels, findfirst(x -> x == [], submodels))
    end
    return submodels
end
    

remove_empty (generic function with 1 method)

In [655]:

using StructuralIdentifiability: parent_ring_change

In [656]:
function ode_aux(ode, submodel)
    #print(ode)
    new_y = copy(ode.y_equations)
    new_x = copy(ode.x_equations)
    for (x,f) in ode.x_equations
        if !(issubset(vars(x),submodel) && issubset(vars(f),submodel))
            delete!(new_x, x)
        end
    end
    
    for (y,f) in ode.y_equations
        if !(issubset(vars(y),submodel) && issubset(vars(f),submodel))
            delete!(new_y, y)
        end
    end
    sub = map(x -> "$x", submodel)
    S, _ = PolynomialRing(QQ, sub)
    fin_x = Dict(parent_ring_change(x, S) => parent_ring_change(f, S) for (x,f) in new_x)
    fin_y = Dict(parent_ring_change(y, S) => parent_ring_change(f, S) for (y,f) in new_y)


    return ODE{fmpq_mpoly}(fin_x, fin_y, Array{fmpq_mpoly, 1}())
end



function submodel2ode(ode2, submodels)
    result = Array{ODE{fmpq_mpoly}, 1}()
    for submodel in submodels
        push!(result, ode_aux(ode2, submodel))
    end
    return result
end





submodel2ode (generic function with 1 method)

In [657]:
function find_submodels(ode)
    graph = construct_graph(ode)
    y = collect(keys(ode.y_equations))
    raw_models = traverse_outputs(graph, y)
    submodels = find_raw_submodels(raw_models, y, graph)
    unions = (search_add_unions(submodels))
    result = remove_empty(union(sort_all(unions)))
    back2ode = submodel2ode(ode, result)
    return back2ode
end

sub = find_submodels(ode)
sub2 = find_submodels(ode2)
sub3 = find_submodels(ode3)


"for (x,f) in sub3[1].x_equations
    println(parent(x))
    println(parent(f))
end"
#println(sub3[2])
#println(sub3)


"for (x,f) in sub3[1].x_equations\n    println(parent(x))\n    println(parent(f))\nend"

In [660]:
"

function  model2ode(ode, submodels)
    result = []
    for submodel in submodels
        new_x = ode.x_equations
        new_y = ode.y_equations
        
        for (x,f) in ode.x_equations
            if !(issubset(vars(x),submodel) && issubset(vars(f),submodel))
                delete!(new_x, x)
            end
        end
        
        for (y,f) in ode.y_equations
            if !(issubset(vars(y),submodel) && issubset(vars(f),submodel))
                delete!(new_y, y)
            end
        end
        sub = map(x -> "$x", submodel)
        S, _ = PolynomialRing(QQ, sub)
        println(new_x)
        println(new_y)
        fin_x = Dict(parent_ring_change(x, S) => parent_ring_change(f, S) for (x,f) in new_x)
        fin_y = Dict(parent_ring_change(y, S) => parent_ring_change(f, S) for (y,f) in new_y)

        println(fin_x)
        println(fin_y)
        res = ODE{fmpq_mpoly}(fin_x, fin_y, Array{fmpq_mpoly, 1}())
        
        push!(result,res)

    end
    return result
end


println(find_submodels(ode3))

new_ode = model2ode(ode3, find_submodels(ode3))
"



LoadError: LoadError: UndefVarError: @x_str not defined
in expression starting at In[660]:20